In [1]:
%matplotlib inline
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
print(torch.__version__)
torch.set_default_tensor_type(torch.FloatTensor)

1.3.1


In [2]:
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [3]:
train_data.shape

(1460, 81)

In [4]:
test_data.shape

(1459, 80)

In [5]:
train_data.iloc[0:4,[1,2,3,-3,-2,-1]]

,MSSubClass,MSZoning,LotFrontage,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,WD,Normal,208500
1,20,RL,80.0,WD,Normal,181500
2,60,RL,68.0,WD,Normal,223500
3,70,RL,60.0,WD,Abnorml,140000


In [6]:
all_features = pd.concat((train_data.iloc[:,1:-1],test_data.iloc[:,1:]))

In [7]:
all_features.shape

(2919, 79)

In [8]:
numeric_features = all_features.dtypes[all_features.dtypes !="object"].index
all_features[numeric_features] = all_features[numeric_features].apply(
        lambda x : (x-x.mean())/(x.std()))
all_features[numeric_features] = all_features[numeric_features].fillna(0)

In [9]:
all_features.shape

(2919, 79)

In [10]:
all_features = pd.get_dummies(all_features, dummy_na =True)

In [11]:
n_train = train_data.shape[0]
train_features = torch.tensor(all_features[:n_train].values, dtype = torch.float32)
test_features = torch.tensor(all_features[n_train:].values , dtype = torch.float32)
train_labels = torch.tensor(train_data.SalePrice.values, dtype = torch.float32).view(-1,1)

In [61]:
def Linear(features_num):
    net = nn.Sequential(
        nn.Linear(features_num,10),
        nn.Linear(10,1)
    )
    return net

In [32]:
loss = nn.MSELoss()
def log_Mse(net , features, labels):
    with torch.no_grad():
        clipped_preds = torch.max(net(features), torch.tensor(1.0))
        rmse = torch.sqrt(2*loss( clipped_preds.log(), labels.log()).mean())
    return rmse

In [38]:
def train(num_epochs, train_features, train_labels, test_features, test_labels, 
                          net,batch_size,lr,weight_decay):
    train_dataset = torch.utils.data.TensorDataset(train_features, train_labels)
    train_l ,test_l =[], []
    optimizer = torch.optim.Adam(params=net.parameters(), lr=lr, weight_decay=weight_decay)
    train_iter = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size, shuffle= True)
    for i in range(num_epochs):
        for x,y in train_iter:
            l = loss(net(x),y)
            optimizer.zero_grad()
            l.backward()
            optimizer.step()
        train_l.append(log_Mse(net, train_features, train_labels))
        if test_features is not None:
            test_l.append(log_Mse(net, test_features, test_labels))
    return train_l,test_l

In [22]:
def get_f_fold(i, k, train_features, train_labels):
    assert k>1
    fold_size = train_features.shape[0]//k
    features_train, labels_train = None,None
    for j in range(k):
        idx = slice(j*fold_size, (j+1)*fold_size)
        features_part,labels_part = train_features[idx,:],train_labels[idx]
        if i == j:
            features_valid,labels_valid = features_part,labels_part
        elif features_train is None:
            features_train,labels_train = features_part,labels_part
        else:
            features_train = torch.cat((features_train,features_part),dim=0)
            labels_train = torch.cat((labels_train,labels_part),dim=0)
    return features_train, labels_train, features_valid,labels_valid

In [44]:
def f_fold(k, x_train, y_train, num_epochs,batch_size,lr,weight_decay):
    train_l_sum,test_l_sum =0,0
    for i in range(k):
        data = get_f_fold(i,k,x_train, y_train)
        net = Linear(x_train.shape[1])
        train_l,test_l = train(num_epochs,*data,net,batch_size,lr,weight_decay)
        train_l_sum += train_l[-1]
        test_l_sum += test_l[-1]
        print('fold %d, train rmse %f, valid rmse %f' % (i, train_l[-1], test_l[-1]))

In [64]:
k, num_epochs, lr, weight_decay, batch_size = 10, 100, 3, 0, 64
f_fold(k,train_features,train_labels,num_epochs,batch_size ,lr,weight_decay)

fold 0, train rmse 0.169255, valid rmse 0.527806
fold 1, train rmse 0.167142, valid rmse 0.268518
fold 2, train rmse 0.199927, valid rmse 0.260834
fold 3, train rmse 0.164061, valid rmse 0.256276
fold 4, train rmse 0.176847, valid rmse 0.288697
fold 5, train rmse 0.182514, valid rmse 0.241463
fold 6, train rmse 0.182010, valid rmse 0.293723
fold 7, train rmse 0.194279, valid rmse 0.204737
fold 8, train rmse 0.168504, valid rmse 0.280256
fold 9, train rmse 0.193296, valid rmse 0.237547


In [67]:
def train_and_pred(train_features, test_features, train_labels, test_data,
                   num_epochs, lr, weight_decay, batch_size):
    net = Linear(train_features.shape[1])
    train_ls, _ = train(num_epochs, train_features, train_labels, None, None,
                        net,batch_size,lr,weight_decay)
    print('train rmse %f' % train_ls[-1])
    preds = net(test_features).detach().numpy()
    test_data['SalePrice'] = pd.Series(preds.reshape(1, -1)[0])
    submission = pd.concat([test_data['Id'], test_data['SalePrice']], axis=1)
    submission.to_csv('./first_housePrice.csv', index=False)

In [68]:
train_and_pred(train_features,test_features,train_labels,test_data,num_epochs,lr,weight_decay,batch_size)

train rmse 0.204109
